# Medical Assistance System

## Description:
This project aims to provide medical assistance by predicting diseases, suggesting remedies, recommending hospitals, and estimating treatment costs and duration based on user input.

## Project Components:
1. **Langchain Module:**
    - Contains modules for OpenAI language models and various utilities.

2. **Main Module (main.py):**
    - Acts as the entry point for the Medical Assistance System.
    - Collects user input about patient details and symptoms.
    - Utilizes sequential chains to predict diseases, suggest remedies, recommend hospitals, and estimate treatment costs and duration.
    - Displays the results to the user.

## Code Implementation

In [1]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import SequentialChain, LLMChain
from langchain.memory import ConversationBufferMemory

### Initialize OpenAI client

In [2]:
mykey ="your key"
client = OpenAI(api_key=mykey)

C:\Users\YASHVI\anaconda3\envs\testingopenai\lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.llms.openai.OpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAI`.
  warn_deprecated(


### Define Prompt Templates

In [12]:
patient_template = PromptTemplate(
    input_variables=['name', 'age', 'symptoms', 'severity', 'city', 'duration'],
    template="I am {name}, {age} years old, experiencing {symptoms} with severity {severity} in {city} for {duration} days. Based on these suggest remedies in few words 10 words"
)

hospital_template = PromptTemplate(
    input_variables=['symptoms', 'city', 'severity'],
    template="Based on symptoms: {symptoms}, severity: {severity}, and city: {city}, recommend only three hospitals names and place for treatment in points and nothing else just the name and place."
)
cost_template = PromptTemplate(
    input_variables=['severity', 'symptoms', 'duration', 'hospital_recommendations'],
    template="Based on the {hospital_recommendations} and additionally severity: {severity}, symptomps:{symptoms}, and duration of symptoms: {duration}, please predict only the anticipated cost in range in rupees (in numbers) for being admitted in those hospitals. Briefly write the name of hospitals and then the predicted costs it will take to be admitted there."
)
time_template = PromptTemplate(
    input_variables=['severity','symptoms','hospital_recommendations'],
    template="Based on the severity: {severity} please predict just the anticipated recovery days/weeks/months range of given symptoms: {symptoms} in few words like 2-3 words just give the time of them"
)
guess_disease = PromptTemplate(
    input_variables=['severity','symptoms'],
    template="Based on the all symptoms: {symptoms} and severity:{severity} predict the disease of the person in one word"
)

### Initialize LLM Chains

In [13]:
guess_chain = LLMChain(llm=client, prompt=guess_disease, output_key="disease")
remedies_chain = LLMChain(llm=client, prompt=patient_template, output_key="remedies")
hospitals_chain = LLMChain(llm=client, prompt=hospital_template, output_key="hospital_recommendations")
cost_chain=LLMChain(llm=client, prompt=cost_template, output_key="cost")
time_chain=LLMChain(llm=client, prompt=time_template, output_key="time")

### Define Sequential Chains

In [14]:
sequential_chain1 = SequentialChain(
    chains=[guess_chain,remedies_chain, hospitals_chain],
    input_variables=['name', 'age', 'symptoms', 'severity', 'city', 'duration'],
    output_variables=["disease","remedies", "hospital_recommendations"]
)
sequential_chain2=SequentialChain(
    chains=[cost_chain,time_chain],
    input_variables=['duration','symptoms','severity', "hospital_recommendations"],
    output_variables=["cost","time"]
)

### User Input

In [15]:
name = input("Name: ")
age = input("Age: ")
symptoms = input("Symptoms: ")
severity = input("Severity: ")
city = input("City: ")
duration = input("Duration of symptoms: ")
response1 = sequential_chain1({"name": name, "age": age, "symptoms": symptoms, "severity": severity, "city": city, "duration": duration})
response2 = sequential_chain2({"symptoms": symptoms, "severity": severity,"duration": duration,"hospital_recommendations":response1["hospital_recommendations"]})

Name:  Yashvi
Age:  19
Symptoms:  sadness, depression headache, loss of apetite
Severity:  high
City:  dwarka, new delhi
Duration of symptoms:  2 months


### Execute Sequential Chains

In [17]:
disease = response1['disease']
remedies = response1["remedies"]
hospital_recommendations = response1["hospital_recommendations"]
cost = response2["cost"]
time_of_treatment = response2["time"]

## Output Results

In [18]:
print('Name:',name)
print('Age:',age)
print('Symptoms:',symptoms)
print('Severity:',severity)
print('City:',city)
print("\nHealth issue:",disease)
print("\nRemedies for",symptoms, "will be:", remedies)
print("\nHospital Recommendations:", hospital_recommendations)
print("\nCost of treatment", cost)
print("\nTime of treatment", time_of_treatment)


Name: Yashvi
Age: 19
Symptoms: sadness, depression headache, loss of apetite
Severity: high
City: dwarka, new delhi

Health issue: 

MajorDepressiveDisorder

Remedies for sadness, depression headache, loss of apetite will be: 

1. Seek therapy or counseling
2. Practice relaxation techniques
3. Exercise regularly
4. Maintain a healthy diet
5. Get enough sleep
6. Engage in hobbies or activities you enjoy
7. Try mindfulness or meditation
8. Consider medication if needed
9. Talk to a trusted friend or family member
10. Seek support from a mental health professional.

Hospital Recommendations: 

1. Max Super Speciality Hospital, Saket - located in South Delhi, about 30 minutes from Dwarka. 
2. Fortis Hospital, Vasant Kunj - located in South West Delhi, about 20 minutes from Dwarka. 
3. BLK Super Speciality Hospital, Pusa Road - located in Central Delhi, about 40 minutes from Dwarka.

Cost of treatment 

1. Max Super Speciality Hospital, Saket - Estimated cost for admission: Rs. 1,50,000 - R